# Nonhardening Plasticity

## Overview

In contrast to elasticity, plasticity describes the deformation of bodies that undergo nonreversible changes of shape in response to applied load.  When a body transitions from elastic to plastic deformation it is said to "yield".  The yield transition point, also known as the **yield strength** is a property of the material and generally changes in response to continued loading.  When the yield strength increases with continued loading it is said to *harden* and the response is described by hardening theories of plasticity.  The simplified nonhardening theory of plasticity describes materials whose yield strength does not change in response to continued loading.  This notebook

- presents an introduction to nonhardening plasticity,
- implements a nonhardening "$J_2$" plasticity model in Matmodlab, and
- verifies the $J_2$ model against analytic solutions.

## See Also

- [User Defined Materials: Introduction](UserMaterials.ipynb)
- [Linear Elastic Material](LinearElastic.ipynb)

<a name='contents'></a>
## Contents

1. <a href='#plastic'>Fundamental Equations</a>
2. <a href='#j2'>$J_2$ Plasticity</a>
3. <a href='#umat.std'>Standard Model Implementation</a>
4. <a href='#umat.ver'>Model Verification</a>

<a name='plastic'></a>
## Fundamental Equations

The mechanical response of a nonhardening plastic material is predicated on the assumption that there exists an **elastic limit**, beyond which stress states are not achievable through normal processes.  The elastic limit is defined by a **yield surface** satisfying

$$
f\left(\pmb{\sigma}\right) = 0
$$

where $f$ is the **yield function** and $\pmb{\sigma}$ the mechanical stress, defined as

$$
\dot{\pmb{\sigma}} = \mathbb{C}{:}\dot{\pmb{\epsilon}}^{\rm e}
$$

where $\mathbb{C}$ is the elastic stiffness and $\dot{\pmb{\epsilon}}^{\rm e}$ the rate of elastic strain.  The rate of strain is commonly regarded as the sum of elastic and plastic parts, giving for the mechanical response

$$
\dot{\pmb{\sigma}} = \mathbb{C}{:}\left(\dot{\pmb{\epsilon}} - \dot{\pmb{\epsilon}}^{\rm p}\right)
$$

where $\dot{\pmb{\epsilon}}^{\rm p}$ is the rate of plastic strain.  Replacing $\dot{\pmb{\epsilon}}^{\rm p}$ with $\dot{\lambda}\pmb{m}$, $\dot{\lambda}$ being the magnitude of $\dot{\pmb{\epsilon}}^{\rm p}$ and $\pmb{m}$ its direction, the mechanical response of the material is

$$
\dot{\pmb{\sigma}} = \mathbb{C}{:}\left(\dot{\pmb{\epsilon}} - \dot{\lambda}\pmb{m}\right)
$$

The solution to the plasticity problem is reduced to determining $\dot{\lambda}$ such that $f\left(\pmb{\sigma}(t)\right)\leq 0 \ \forall t>0$

### Solution Process

Given the current state of stress $\pmb{\sigma}_n$, the solution to the plasticity problem begins with the hypothesis that the entire strain increment is elastic:

$$
\pmb{\sigma}_{n+1} \stackrel{?}{=} \pmb{\sigma}_{n} + \mathbb{C}{:}\dot{\pmb{\epsilon}}dt = \pmb{\sigma}^{\rm test}
$$

where the descriptor "test" is used to signal the fact that at this point $\pmb{\sigma}^{\rm test}$ is merely a hypothesized solution.  The hypothesis is validated if $\pmb{\sigma}^{\rm test}$ satisfies the **yield condition** 

$$f\left(\pmb{\sigma}^{\rm test}\right)\leq 0$$

so that $\pmb{\sigma}_{n+1}=\pmb{\sigma}^{\rm test}$.  

If instead the hypothesis is *falsefied*, i.e., the predicted test stress falls outside of the yield surface defined by $f=0$, the plasticity problem, 

$$\begin{align}
\pmb{\sigma}_{n+1} = \pmb{\sigma}_{n} + \mathbb{C}{:}\left(\dot{\pmb{\epsilon}} - \dot{\lambda}\pmb{m}\right)dt &= \pmb{\sigma}^{\rm trial} - \dot{\lambda}\pmb{A}dt\\
f\left(\pmb{\sigma}^{\rm trial} - \dot{\lambda}\pmb{A}dt\right) &= 0
\end{align}$$

where $\pmb{A}=\mathbb{C}{:}\pmb{m}$, is solved.  $\dot{\pmb{\sigma}}^{\rm trial}=\mathbb{C}{:}\dot{\pmb{\epsilon}}$ is distinguished from $\dot{\pmb{\sigma}}^{\rm test}$ in that for stress driven problems $\dot{\pmb{\sigma}}^{\rm trial}$ is not necessarily known because the strain rates $\dot{\epsilon}$ are not known.

The unknown scalar $\dot{\lambda}$ is determined by noting the following observation: if $f\left(\pmb{\sigma}_{n}\right)=0$ and, after continued loading, $f\left(\pmb{\sigma}_{n+1}\right)=0$, the rate of change of $f$ itself must be zero.  Thus, by the chain rule,

$$\begin{align}
\dot{f}{\left(\pmb{\sigma}\right)}
    &=\frac{df}{d\pmb{\sigma}}{:}\dot{\pmb{\sigma}}\\
     &=\frac{df}{d\pmb{\sigma}}{:}\left(\mathbb{C}{:}\dot{\epsilon}-\dot{\lambda}\pmb{A}dt\right)=0
\end{align}$$

which is known as the **consistency condition** and $\dot{\lambda}$ the **consistency parameter**.  The consistency condition is equivalent to the statement that $\dot{\lambda}\dot{f}=0$.  The consistency conditions can be shown to be equivalent to the Karush-Kuhn-Tucker conditions

$$
   \dot{\lambda} \ge 0 ~,~~ f \le 0~,~~ \dot{\lambda}\,f = 0 \,.
$$

Letting

$$
\pmb{n} = \frac{df}{d\pmb{\sigma}}\Big/\Big\lVert\frac{df}{d\pmb{\sigma}}\Big\rVert
$$

the preceding equation can be solved $\dot{\lambda}$, giving

$$
\dot{\lambda} 
   = \frac{\pmb{n}{:}\mathbb{C}{:}\dot{\epsilon}}{\pmb{n}{:}\pmb{A}}
$$

Substituting $\dot{\lambda}$ in to the expression for stress rate gives

$$\begin{align}
\dot{\pmb{\sigma}} 
   &= \mathbb{C}{:}\dot{\pmb{\epsilon}} - \frac{\pmb{n}{:}\mathbb{C}{:}\dot{\epsilon}}{\pmb{n}{:}\pmb{A}}\pmb{A}\\
   &= \mathbb{C}{:}\dot{\pmb{\epsilon}} - \frac{1}{\pmb{n}{:}\pmb{A}}\pmb{Q}\pmb{A}{:}\dot{\pmb{\epsilon}}\\
   &= \mathbb{D}{:}\dot{\pmb{\epsilon}}
\end{align}$$

where

$$
\mathbb{D} = \mathbb{C} - \frac{1}{\pmb{n}{:}\pmb{A}}\pmb{Q}\pmb{A}
$$

The stress rate is then integrated through time to determine $\pmb{\sigma}$

### Integration Procedure

It can be shown that there exists a scalar $\Gamma$ such that

$$\pmb{\sigma}_{n+1} = \pmb{\sigma}^{\rm trial} - \Gamma\pmb{A}$$

$\Gamma$ is determined by satisfying the yield condition.  In other words, $\Gamma$ is the solution to

$$f\left(\pmb{\sigma}^{\rm trial} - \Gamma\pmb{A}\right)=0$$

The unknown $\Gamma$ is found such that $f\left(\pmb{\sigma}(\Gamma)\right)=0$.  The solution can be found by solving the preceding equation iteratively by applying Newton's method so that

$$
\Gamma^{i+1} = \Gamma^i + \frac{f\left(\pmb{\sigma}(\Gamma^{n})\right)}{(df/d\pmb{\sigma})\big|_{\Gamma^i}{:}\pmb{A}}
$$

When $\Gamma^{i+1}-\Gamma^i<\epsilon$, where $\epsilon$ is a small number, the iterations are complete and the updated stress can be determined.

Note that the scalar $\Gamma$ is also equal to $\Gamma=\dot{\lambda}dt$, but since $\dot{\lambda}=0$ for elastic loading, $\Gamma=\dot{\lambda}dt^p$, where $dt^p$ is the plastic part of the time step.  This gives $\Gamma$ the following physical interpretation: it is the magnigute of the total plastic strain increment.

#### Alternative Iterative method

Brannon argues that redefining $\Gamma$ such that

$$\pmb{\sigma}_{n+1} = \pmb{\sigma}^{\rm trial} + \Gamma\pmb{A}$$

can lead to fast convergence for yield surfaces with considerable curvature [2002, Radial Return document].  To update the stress, begin Newton iterations with $\Gamma=0$ and compute the improved estimate as

$$
\Gamma^{i+1} = -\frac{f\left(\pmb{\sigma}(\Gamma^{i})\right)}{(df/d\pmb{\sigma})\big|_{\Gamma^i}{:}\pmb{A}}
$$

when $\Gamma^{i+1}<\epsilon$ the Newton procedure is complete.

## $J_2$ Plasticity

The equations developed thus far are general in the sense that they apply to any material that can be modeled by nonhardening plasticity.  The equations are now specialized to the case of isotropic hypoelasticity and $J_2$ plasticity by defining

$$\begin{align}
\dot{\pmb{\sigma}} &= 3K\,\mathrm{iso}\dot{\pmb{\epsilon}}^{\rm e} + 2G\,\mathrm{dev}\dot{\pmb{\epsilon}}^{\rm e} \\
f\left(\pmb{\sigma}\right) &= \sqrt{J_2} - k
\end{align}
$$

where $K$ and $G$ are the bulk and shear moduli, respectively, and $J_2$ is the second invariant of the stress deviator, defined as

$$J_2 = \frac{1}{2}\pmb{s}{:}\pmb{s}, \quad \pmb{s} = \pmb{\sigma} - \frac{1}{3}\mathrm{tr}\left(\pmb{\sigma}\right)\pmb{I}$$

Additionally, we adopt the assumption of an **associative flow rule** wherein $\pmb{m}=\pmb{n}$.  Accordingly,

$$\begin{align}
\frac{df}{d\pmb{\sigma}}&=\frac{1}{2\sqrt{J_2}}\pmb{s}, &\pmb{n}=\frac{1}{\sqrt{2J_2}}\pmb{s} \\
\pmb{A}&=\frac{2G}{\sqrt{2J_2}}\pmb{s}, &\pmb{Q}=\frac{2G}{\sqrt{2J_2}}\pmb{s}
\end{align}$$

### Required Parameters

The model as described above requires at minimum 3 parameters: 2 independent elastic moduli and a yield strength measure.  Commonly, the yield strength in tension $Y$ is chosen in lieu of the yield strength in shear $k$.  The conversion from $Y$ to $k$, as required by the model development, is found from evaluating the yield criterion for the case of uniaxial tension, where the stress and its deviator are

$$
\pmb{\sigma} =  \begin{bmatrix}\sigma_{\rm ax}&&\\& 0&\\&& 0\end{bmatrix}, \quad
\pmb{s} =  \frac{1}{3}\begin{bmatrix}2\sigma_{\rm ax}&&\\& -\sigma_{\rm ax}&\\&& -\sigma_{\rm ax}\end{bmatrix}
$$

Accordingly, 

$$J_2 = \frac{1}{2}\pmb{s}{:}\pmb{s} = \frac{1}{3}\sigma_{\rm ax}^2$$

evaluating the yield function leads to

$$\sqrt{J_2} = \frac{1}{\sqrt{3}}Y = k$$

### Optional Solution Dependent Variables

The simple $J_2$ plasticity model described does not require the storage and tracking of solution dependent variables.  However, the equivalent plastic strain is often stored for output purposes.  The equivalent plastic strain is defined so that the plastic work

$$W^p = \pmb{\sigma}{:}\dot{\pmb{\epsilon}}^p=\sigma_{eq}\dot{\epsilon}^p_{eq}$$

where $\sigma_{eq}$ is the equivalent von Mises stress $\sigma_{eq} = \sqrt{\frac{3}{2}\pmb{s}{:}\pmb{s}}$,  requiring that

$$\dot{\epsilon}^p_{eq} = \sqrt{\frac{2}{3}}\lVert{\rm dev}\left(\dot{\pmb{\epsilon}}^p\right)\rVert$$

<a name='umat.std'></a>
## Model Implementation

Material models implemented as subclasses of the `MaterialModel` class are referred to as "standard" materials.  Minimally, the materials installed as standard materials must define

- `name`: *class attribute*

   Used for referencing the material model in the `MaterialPointSimulator`.
   
- `param_names`: *class method*

   Used by the the `MaterialPointSimulator` for parsing input parameter names and assembling a material parameters array.
   
- `update_state`: *instance method*

   Updates the material stress, stiffness (optional), and state dependent variables to the end of the time increment.  If the stiffness is returned as `None`, Matmodlab will determine it numerically.
  
Additionally, materials requiring solution-dependent variables to be allocated and stored by Matmodlab must define the optional `setup` method

- `setup`: *instance method [optional]*

   Checks goodness of user input and requests storage allocation for state dependent variables.  The return values from this method are `sdv_names` and `sdv_vals`, which are lists of the solution-dependent variable names and initial values, respectively.
   
In the example below, in addition to some standard functions imported from `Numpy`, several helper functions are imported from various locations in Matmodlab:

- `matmodlab.utils.tensor`

   - `iso`, `dev`: computes the isotropic and deviatoric parts of a second-order tensor
   - `mag`: computes the magnitude $\left(\lVert x_{ij} \rVert=\sqrt{x_{ij}x_{ij}}\right)$ of a second-order tensors
   - `ddot`, `dyad`: computes the double dot product and dyadic product of second-order tensors
- `matmodlab.constants`
   - `VOIGT`: mulitplier for converting tensor strain components to engineering strain components
   - `ROOT3`: $\sqrt{3}$
   - `TOLER`: A small number, used as a tolerance
   

### A Note on Implementation

For the simplistic $J_2$ plasticity model, it is a simple task to determine the analytic response and avoid Newton iterations.  In the model implemented, Newton iterations are used to verify the algorithm.

### Loading the Material Model

Once defined in a computational cell, the material model is loaded in to Matmodlab through the `load_material` function using the `std_material` keyword.

In [1]:
%matmodlab

BokehJS successfully loaded.

Populating the interactive namespace from matmodlab and bokeh


In [2]:
import logging
from numpy import dot, zeros, ix_, eye
from matmodlab.mmd.material import MaterialModel
from matmodlab.utils.tensor import dyad, ddot, iso, dev, mag
from matmodlab.constants import VOIGT, ROOT3, ROOT2, TOLER

class UserPlastic(MaterialModel):
    name = "uplastic-std"

    @classmethod
    def param_names(cls, n):
        """Parameters are: E (Young's modulus), Nu (Poisson's ratio), 
        and Y (yield strength in tension)
        
        """
        return ['E', 'Nu', 'Y']

    def setup(self, **kwargs):
        """Set up the Elastic material

        """
        logger = logging.getLogger('matmodlab.mmd.simulator')

        # Check inputs
        E = self.parameters['E']
        Nu = self.parameters['Nu']
        Y = self.parameters['Y']
        errors = 0
        if E <= 0.0:
            errors += 1
            logger.error("Young's modulus E must be positive")
        if Nu > 0.5:
            errors += 1
            logger.error("Poisson's ratio > .5")
        if Nu < -1.0:
            errors += 1
            logger.error("Poisson's ratio < -1.")
        if Nu < 0.0:
            logger.warn("#---- WARNING: negative Poisson's ratio")
        if Y < 0:
            errors += 1
            logger.error('Yield strength must be positive')
        if Y < 1e-12:
            # zero strength -> assume the user wants elasticity
            logger.info('Zero strength detected, setting it to a larg number')
            self.parameters['Y'] = 1e60
            
        if errors:
            raise ValueError("stopping due to previous errors")
            
        # At this point, the parameters have been checked.  Now request
        # allocation of solution dependent variables.  The only variable
        # is the equivalent plastic strain
        
        sdv_names = ['EP.Equiv']
        sdv_vals = [0]
        
        return sdv_names, sdv_vals

    def update_state(self, time, dtime, temp, dtemp, energy, rho, F0, F,
        stran, d, elec_field, stress, X, **kwargs):
        """Compute updated stress given strain increment"""

        #  material properties
        Y = self.parameters['Y']
        E = self.parameters['E']
        Nu = self.parameters['Nu']
        
        # Input parameter is yield in tension -> convert to yield in shear
        k = Y / ROOT3

        # Get the bulk, shear, and Lame constants
        K = E / 3. / (1. - 2. * Nu)
        G = E / 2. / (1. + Nu)

        K3 = 3. * K
        G2 = 2. * G
        G3 = 3. * G
        Lam = (K3 - G2) / 3.

        # elastic stiffness
        C = zeros((6,6))
        C[ix_(range(3), range(3))] = Lam
        C[range(3),range(3)] += G2
        C[range(3,6),range(3,6)] = G

        # Trial stress
        de = d * dtime
        T = stress + dot(C, de) 
   
        # check yield
        S = dev(T)
        RTJ2 = mag(S) / ROOT2
        f = RTJ2 - k

        if f <= TOLER:
            # Elastic loading, return what we have computed
            return T, X, C
 
        # Calculate the flow direction, projection direction
        M = S / ROOT2 / RTJ2
        N = S / ROOT2 / RTJ2
        A = 2 * G * M
        
        # Newton iterations to find Gamma
        Gamma = 0
        Ttrial = T.copy()
        for i in range(20):
        
            # Update all quantities
            dGamma = f * ROOT2 / ddot(N, A)
            Gamma += dGamma
            
            T = Ttrial - Gamma * A
            S = dev(T)
            RTJ2 = mag(S) / ROOT2
            f = RTJ2 - k
            
            # Calculate the flow direction, projection direction
            M = S / ROOT2 / RTJ2
            N = S / ROOT2 / RTJ2
            A = 2 * G * M
            Q = 2 * G * N
            
            if abs(dGamma + 1.) < TOLER + 1.:
                break
            
        else:
            raise RuntimeError('Newton iterations failed to converge')
    
        # Elastic strain rate and equivalent plastic strain
        dT = T - stress
        dep = Gamma * M
        dee = de - dep
        deqp = ROOT2 / ROOT3 * Gamma
        
        # Elastic stiffness
        D = C - 1 / ddot(N, A) * dyad(Q, A)
        
        # Equivalent plastic strain
        X[0] += deqp
        
        return T, X, D
        
load_material(std_material=UserPlastic)

<a name='umat.ver'></a>
## Verification

Exercising the elastic model through a path of uniaxial stress should result in the slope of axial stress vs. axial strain being equal to the input parameter `E` for the elastic portion.  The maximum stress should be equal to the input parameter `Y`.

**Note:** the input parameters to a standard material are given as a dictionary of `name:value` pairs for each paramter.  Parameters not specified are initialized to a value of zero.

In [3]:
mps1 = MaterialPointSimulator('uplastic-std')
mps1.Material('uplastic-std', {'E': 10e6, 'Nu': .333, 'Y': 40e3})
mps1.MixedStep(components=(.02, 0, 0), descriptors='ESS', frames=50)

i = where((mps1.E.XX > 0.) & (mps1.E.XX < .005))
E = mps1.S.XX[i] / mps1.E.XX[i]
assert allclose(E[0], 10e6, atol=1e-3, rtol=1e-3)
assert amax(mps1.S.XX) - 40e3 < 1e-6
show(mps1.plot('E.XX', 'S.XX'))

In [4]:
mps2 = MaterialPointSimulator('uplastic-std')
mps2.Material('uplastic-std', {'E': 10e6, 'Nu': .333, 'Y': 40e3})
mps2.MixedStep(components=(.02, 0, 0), descriptors='EEE', frames=50)

i = where((mps2.E.XX > 0.) & (mps2.E.XX < .005))
#H = mps1.S.XX[i] / mps1.E.XX[i]
#assert allclose(H, 10e6, atol=1e-3, rtol=1e-3)
#assert amax(mps1.S.XX) - 40e3 < 1e-6
show(mps2.plot('E.XX', 'S.XX'))

In [5]:
mps3 = MaterialPointSimulator('uplastic-std')
mps3.Material('uplastic-std', {'E': 10e6, 'Nu': .333, 'Y': 40e3})
mps3.MixedStep(components=(.02, 0, 0), descriptors='ESS', frames=50)
mps3.MixedStep(components=(0, 0, 0), descriptors='ESS', frames=50)
mps3.MixedStep(components=(-.02, 0, 0), descriptors='ESS', frames=50)
mps3.MixedStep(components=(0, 0, 0), descriptors='ESS', frames=50)

mps4 = MaterialPointSimulator('uplastic-std')
mps4.Material('uplastic-std', {'E': 10e6, 'Nu': .333, 'Y': 40e3})
mps4.MixedStep(components=(.02, 0, 0), descriptors='EEE', frames=50)
mps4.MixedStep(components=(0, 0, 0), descriptors='EEE', frames=50)
mps4.MixedStep(components=(-.02, 0, 0), descriptors='EEE', frames=50)
mps4.MixedStep(components=(0, 0, 0), descriptors='EEE', frames=50)

In [6]:
show(gridplot([[mps3.plot('E.XX', 'S.XX', plot_width=350, plot_height=350), 
           mps4.plot('E.XX', 'S.XX', color='red', plot_width=350, plot_height=350)]]));